Before starting to work on the data, we need to obtain the data from the data sources. In order to do so I used web scraping technique and pandas read_csv function to collect the data.

# Web Scraping - daily_treasury_yield_curve_rate

In [ ]:
import requests
from lxml import html
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [ ]:
URL = 'https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData'
COLUMNS = ['Date','1 Mo','2 Mo','3 Mo','6 Mo','1 Yr','2 Yr','3 Yr','5 Yr','7 Yr','10 Yr','20 Yr','30 Yr']

In [ ]:
def process_data(soup):
    data = soup.find_all('m:properties')[0]
    data = data.text
    data = data.split('\n')
    data = data[2:-2]
    return data

def create_new_row(data,columns):
    new_row = dict()
    
    for idx,(key,value) in enumerate(zip(columns,data)):
        if idx == 0:
            value = datetime.strptime(value, '%Y-%m-%dT%H:%M:%S')
        else:
            value = round(float(value),2) if value != '' else 0
                
        new_row[key] = value
        
    return new_row

def get_data_from_link(link,columns):
    sauce = urllib.request.urlopen(link).read()
    soup = bs(sauce,'xml')
    data = process_data(soup)
    new_row = create_new_row(data,columns)
    return new_row

sauce = urllib.request.urlopen(URL).read()
soup = bs(sauce,'xml')
df = pd.DataFrame(columns=COLUMNS)
for idx,link in enumerate(soup.find_all('id')):
    if idx>0:
        daily_yield_curve_rate_data = get_data_from_link(link.text,COLUMNS)
        df = df.append(daily_yield_curve_rate_data,ignore_index=True)

df.to_csv('daily_treasury_yield_curve_rate.csv')


# Web Scraping - BAA10Y

In [ ]:
import pandas as pd
import io
import requests

URL = r'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=BAA10Y&scale=left&cosd=1986-01-02&coed=2019-12-03&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2009-06-01&line_index=1&transformation=lin&vintage_date=2019-12-05&revision_date=2019-12-05&nd=1990-02-02'
c = pd.read_csv(URL)
print(c)